In [49]:
# fullwidth notebook cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [50]:
import tensorflow as tf

# for reading & slicing data
import pandas as pd

# for data preprocessing
from sklearn.preprocessing import MinMaxScaler

import numpy as np

In [51]:
data_path = 'data/wrap_data_test_1.csv'
data_df = pd.read_csv(data_path, dtype = float)

training_data_df = data_df.sample(frac = 0.8)
testing_data_df = data_df.loc[~data_df.index.isin(training_data_df.index)]

X_training = training_data_df.drop('tau', axis = 1).values
Y_training = training_data_df[['tau']].values

X_testing = testing_data_df.drop('tau', axis = 1).values
Y_testing = testing_data_df[['tau']].values

X_scaler = MinMaxScaler(feature_range = (0,1))
Y_scaler = MinMaxScaler(feature_range = (0,1))

X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

number_of_inputs = X_scaled_training.shape[1]
number_of_outputs = Y_scaled_training.shape[1]

learning_rate = 0.0015
training_epochs = 200
display_step = 2

In [52]:
layer_1_node_vals = np.arange(50,550,50)
layer_2_node_vals = np.arange(50,550,50)
layer_3_node_vals = np.arange(50,550,50)

log_paths = []
for layer_1_nodes in layer_1_node_vals:
    for layer_2_nodes in layer_2_node_vals:
        for layer_3_nodes in layer_3_node_vals:
            
            log_path = str(layer_1_nodes) + "_" + str(layer_2_nodes) +  "_" + str(layer_3_nodes)
                
            log_paths.append(log_path)

In [53]:
random_layers = np.random.choice(log_paths, size = 10)

In [72]:
def f(layer_1_nodes,layer_2_nodes,layer_3_nodes, logdir = ''):
    
    tf.reset_default_graph()
    
    # Input Layer
    with tf.variable_scope('input'):
        X  = tf.placeholder(tf.float32, shape = (None, number_of_inputs))
        
    # Layer 1
    with tf.variable_scope('layer_1'):
        
        biases = tf.get_variable(name = "biases1",
                                 shape = [layer_1_nodes],
                                 initializer = tf.zeros_initializer())
        
        weights = tf.get_variable(name = "weights1",
                                  shape  = [number_of_inputs, layer_1_nodes],
                             initializer = tf.contrib.layers.xavier_initializer())
    
    
        layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)
        
    # Layer 2
    with tf.variable_scope('layer_2'):
        
        biases = tf.get_variable(name = "biases2",
                                 shape = [layer_2_nodes],
                                 initializer = tf.zeros_initializer())
        
        weights = tf.get_variable(name = "weights2",
                                  shape  = [layer_1_nodes, layer_2_nodes],
                             initializer = tf.contrib.layers.xavier_initializer())
    
    
        layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)
        
    # Layer 3
    with tf.variable_scope('layer_3'):
        
        biases = tf.get_variable(name = "biases3",
                                 shape = [layer_3_nodes],
                                 initializer = tf.zeros_initializer())
        
        weights = tf.get_variable(name = "weights3",
                                  shape  = [layer_2_nodes, layer_3_nodes],
                             initializer = tf.contrib.layers.xavier_initializer())
    
    
        layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)
        
    # Output layer
    
    with tf.variable_scope('output'):
        
        biases = tf.get_variable(name = "biases_out",
                                 shape = [number_of_outputs],
                                 initializer = tf.zeros_initializer())
        
        weights = tf.get_variable(name = "weights_out",
                                  shape  = [layer_3_nodes, number_of_outputs],
                             initializer = tf.contrib.layers.xavier_initializer())
    
    
        prediction = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)
        
    with tf.variable_scope('cost'):
        
        Y = tf.placeholder(tf.float32, shape = (None, 1))
        cost = tf.reduce_mean(tf.squared_difference(prediction, Y))
    
        
    with tf.variable_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate, epsilon=1e-05).minimize(cost)
        
    with tf.variable_scope('logging'):
        tf.summary.scalar('current_cost', cost)
        summary = tf.summary.merge_all()
        
    saver = tf.train.Saver()
    
    # initialize a session to run TF operations
    with tf.Session() as session:

        # Run the global variable initilizer to init all variables and layers
        session.run(tf.global_variables_initializer())

        # Create log writers to record training progress
        # Store training and testing data separately
        training_writer = tf.summary.FileWriter('logs/' + logdir + random_layer + '/training', session.graph)
        testing_writer = tf.summary.FileWriter('logs/' + logdir + random_layer + '/testing', session.graph)


        # Run the optimizer over and over to train the network
        # One epoch is one full run through the training data set
        for epoch in range(training_epochs):

            # Feed in the training data and do one stepf NN training
            session.run(optimizer,
                        feed_dict = {X: X_scaled_training, Y: Y_scaled_training})

            # Every display_step steps, log our process
            if epoch % 10 == 0:

                training_feed = {X: X_scaled_training, Y: Y_scaled_training}
                training_cost, training_summary = session.run([cost, summary],
                                                        feed_dict = training_feed)

                testing_feed =  {X: X_scaled_testing, Y: Y_scaled_testing}
                testing_cost, testing_summary = session.run([cost, summary],
                                                         feed_dict = testing_feed)

                print(epoch, training_cost, testing_cost)

                # write the current status to the log files
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            testing_writer.flush()
            training_writer.flush()
    #         print('Training done')

        final_training_cost = session.run(cost,
                                          feed_dict = {X: X_scaled_training,
                                                       Y: Y_scaled_training})

        final_testing_cost = session.run(cost,
                                         feed_dict = {X: X_scaled_testing,
                                                      Y: Y_scaled_testing})

#         print('Final Training Cost: {}'.format(final_training_cost))
#         print('Final Testing Cost: {}'.format(final_testing_cost))

        model_save_location = 'logs/' + logdir + random_layer + 'model' + '/trained_model.ckpt'
        save_path = saver.save(session, model_save_location)

In [43]:
for i, random_layer in enumerate(random_layers):
    
    nodes = np.fromstring(random_layer, sep = '_')
    layer_1_nodes, layer_2_nodes, layer_3_nodes = int(nodes[0]), int(nodes[1]), int(nodes[2])
    f(layer_1_nodes,layer_2_nodes,layer_2_nodes)

0 4.950407 4.9488955
10 0.092782214 0.09390253
20 0.08792446 0.08885831
30 0.08650418 0.08705993
40 0.08539292 0.08592592
50 0.0849574 0.085475385
60 0.084172525 0.08471407
70 0.08310406 0.083835036
80 0.081669465 0.08272992
90 0.079245426 0.080549836
100 0.07623921 0.07754141
110 0.07292085 0.0741587
120 0.06884875 0.07018775
130 0.06389319 0.06537057
140 0.05873826 0.060682923
150 0.054106556 0.055812694
160 0.052716155 0.05485922
170 0.045441926 0.04682956
180 0.041895043 0.04330854
190 0.043108482 0.044810675
200 0.038088385 0.039587583
210 0.032992378 0.03512038
220 0.028579516 0.030302636
230 0.034929328 0.037163075
240 0.028512143 0.030854516
250 0.02178 0.023638528
260 0.018994648 0.020786129
270 0.015854888 0.018019589
280 0.021765104 0.02351233
290 0.013753706 0.015376572
300 0.010465577 0.012149467
310 0.010093743 0.011804394
320 0.00965171 0.011041803
330 0.0070971046 0.008507437
340 0.0060792635 0.007344389
350 0.00786021 0.009211924
360 0.0053348816 0.0065269447
370 0.004

370 0.0186172 0.021781616
380 0.018060546 0.021301895
390 0.017421117 0.021007175
400 0.016851906 0.020317784
410 0.019140022 0.022277098
420 0.016801441 0.020231357
430 0.016759617 0.020446239
440 0.01590882 0.019525593
450 0.016878007 0.020299887
460 0.016504101 0.020329967
470 0.01531435 0.019016078
480 0.016500542 0.020364428
490 0.015513591 0.019162606
Training done
Final Training Cost: 0.014979911968111992
Final Testing Cost: 0.018741684034466743
0 16.372572 16.33058
10 0.33241737 0.32945666
20 0.33241737 0.32945666
30 0.33241737 0.32945666
40 0.33241737 0.32945666
50 0.33241737 0.32945666
60 0.33241737 0.32945666
70 0.33241737 0.32945666
80 0.33241737 0.32945666
90 0.33241737 0.32945666
100 0.33241737 0.32945666
110 0.33241737 0.32945666
120 0.33241737 0.32945666
130 0.33241737 0.32945666
140 0.33241737 0.32945666
150 0.33241737 0.32945666
160 0.33241737 0.32945666
170 0.33241737 0.32945666
180 0.33241737 0.32945666
190 0.33241737 0.32945666
200 0.33241737 0.32945666
210 0.33241

In [48]:
layer_1_node_vals = np.arange(15,90,15)
layer_2_node_vals = np.arange(300,500,50)
layer_3_node_vals = np.arange(200,450,50)

log_paths = []
for layer_1_nodes in layer_1_node_vals:
    for layer_2_nodes in layer_2_node_vals:
        for layer_3_nodes in layer_3_node_vals:
            
            log_path = str(layer_1_nodes) + "_" + str(layer_2_nodes) +  "_" + str(layer_3_nodes)
                
            log_paths.append(log_path)
            
random_layers = np.random.choice(log_paths, size = 10)

for i, random_layer in enumerate(random_layers):
    print(i)
    
    nodes = np.fromstring(random_layer, sep = '_')
    layer_1_nodes, layer_2_nodes, layer_3_nodes = int(nodes[0]), int(nodes[1]), int(nodes[2])
    f(layer_1_nodes,layer_2_nodes,layer_2_nodes)

0
0 1.092518 1.0838659
10 0.108982146 0.106353134
20 0.09020366 0.088460326
30 0.08680518 0.0853833
40 0.08346019 0.08243761
50 0.08148037 0.08079836
60 0.077842735 0.07809026
70 0.073062144 0.07414793
80 0.06746452 0.06923722
90 0.0619455 0.06465176
100 0.0529656 0.056200363
110 0.042191207 0.045547165
120 0.050681308 0.054325454
130 0.03105121 0.033514332
140 0.018628463 0.021141188
150 0.013243215 0.015516875
160 0.009899633 0.011699475
170 0.008121009 0.009827097
180 0.0075607575 0.009084788
190 0.005954163 0.007558496
Training done
Final Training Cost: 0.004937790334224701
Final Testing Cost: 0.006422697100788355
1
0 0.6492391 0.64567804
10 0.09879422 0.09854036
20 0.091977045 0.09185806
30 0.08558762 0.085616134
40 0.08297661 0.08362246
50 0.078740925 0.07986132
60 0.071908355 0.073281206
70 0.063247174 0.06534017
80 0.052824393 0.055335175
90 0.045228273 0.04708666
100 0.03900267 0.040696442
110 0.035284553 0.03722091
120 0.032055214 0.03375241
130 0.02923407 0.03145053
140 0.02

In [57]:
layer_1_node_vals = np.arange(15,90,15)
layer_2_node_vals = np.arange(300,500,50)
layer_3_node_vals = np.arange(200,450,50)

log_paths = []
for layer_1_nodes in layer_1_node_vals:
    for layer_2_nodes in layer_2_node_vals:
        for layer_3_nodes in layer_3_node_vals:
            
            log_path = str(layer_1_nodes) + "_" + str(layer_2_nodes) +  "_" + str(layer_3_nodes)
                
            log_paths.append(log_path)
            
random_layers = np.random.choice(log_paths, size = 10)

for i, random_layer in enumerate(random_layers):
    print(i)
    
    nodes = np.fromstring(random_layer, sep = '_')
    layer_1_nodes, layer_2_nodes, layer_3_nodes = int(nodes[0]), int(nodes[1]), int(nodes[2])
    f(layer_1_nodes,layer_2_nodes,layer_2_nodes, logdir = 'Run_2/')

0
0 0.77498907 0.766237
10 0.09732775 0.093258716
20 0.09164944 0.089056075
30 0.08678006 0.08460295
40 0.0853721 0.08353206
50 0.08372381 0.082156554
60 0.08157689 0.08042889
70 0.07996355 0.07913939
80 0.07843379 0.077443115
90 0.0774434 0.076525755
100 0.076402836 0.07567772
110 0.07583565 0.07534523
120 0.07510067 0.07481857
130 0.07463617 0.074657716
140 0.07351124 0.073775515
150 0.072000355 0.07243667
160 0.07026247 0.07099222
170 0.06803308 0.06946418
180 0.0663775 0.06843388
190 0.06284814 0.065232635
1
0 0.42659283 0.4205511
10 0.09563686 0.09650145
20 0.088516675 0.08803746
30 0.08608386 0.08558406
40 0.084617816 0.0836542
50 0.083297074 0.082246006
60 0.082822345 0.08180079
70 0.08241768 0.08133022
80 0.08191017 0.080967315
90 0.08120527 0.08032438
100 0.08012067 0.079333365
110 0.078348815 0.07771081
120 0.07546703 0.07511015
130 0.07101743 0.07097125
140 0.06487825 0.064920515
150 0.058771636 0.058537636
160 0.052984554 0.052843552
170 0.049992777 0.050095025
180 0.047875

In [62]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
event_acc = EventAccumulator('logs/Run_1/50_400_300/testing/events.out.tfevents.1527222922.Andrews-MacBook.local')
event_acc.Reload()
# Show all tags in the log file
print(event_acc.Tags())


{'images': [], 'audio': [], 'histograms': [], 'scalars': ['logging/current_cost'], 'distributions': [], 'tensors': [], 'graph': True, 'meta_graph': True, 'run_metadata': []}


In [73]:
layer_1_node_vals = np.arange(15,90,5)
layer_2_node_vals = np.arange(300,510,5)
layer_3_node_vals = np.arange(200,460,5)

log_paths = []
for layer_1_nodes in layer_1_node_vals:
    for layer_2_nodes in layer_2_node_vals:
        for layer_3_nodes in layer_3_node_vals:
            
            log_path = str(layer_1_nodes) + "_" + str(layer_2_nodes) +  "_" + str(layer_3_nodes)
                
            log_paths.append(log_path)
            
random_layers = np.random.choice(log_paths, size = 20)

for i, random_layer in enumerate(random_layers):
    print(i)
    
    nodes = np.fromstring(random_layer, sep = '_')
    layer_1_nodes, layer_2_nodes, layer_3_nodes = int(nodes[0]), int(nodes[1]), int(nodes[2])
    f(layer_1_nodes,layer_2_nodes,layer_2_nodes, logdir = 'Run_4/')

0
0 0.7841674 0.7849297
10 0.14716199 0.14669153
20 0.09487278 0.09344031
30 0.08514947 0.08347885
40 0.08381155 0.08220157
50 0.08310977 0.081511
60 0.08297729 0.08136915
70 0.082795136 0.081228055
80 0.082740076 0.081145175
90 0.08272541 0.08114774
100 0.082718104 0.08113066
110 0.08271497 0.08113024
120 0.08271357 0.081131
130 0.082711354 0.08112723
140 0.082709834 0.08112593
150 0.0827084 0.08112529
160 0.08270721 0.08112431
170 0.082705945 0.08112306
180 0.08270476 0.08112198
190 0.08270347 0.08112082
1
0 0.7929619 0.7955028
10 0.10155539 0.103805326
20 0.09930985 0.10043191
30 0.08828131 0.08849667
40 0.08401255 0.08363603
50 0.08315819 0.08224492
60 0.08271019 0.08146532
70 0.08245946 0.0812224
80 0.08226578 0.080845386
90 0.08206681 0.08063574
100 0.08184552 0.08036759
110 0.08153705 0.08008336
120 0.08110127 0.079707265
130 0.080407925 0.079123676
140 0.07936334 0.07824754
150 0.07783441 0.077020876
160 0.075619444 0.07520778
170 0.07234781 0.07248872
180 0.0676082 0.06848089


160 0.011846073 0.014223602
170 0.01115736 0.013368486
180 0.010195303 0.0125300195
190 0.009778306 0.011979475
16
0 2.834365 2.841525
10 0.10211123 0.10041694
20 0.092757136 0.09120726
30 0.0851514 0.083443224
40 0.08308894 0.0818871
50 0.0795648 0.07856903
60 0.07578369 0.07530424
70 0.07105794 0.07136026
80 0.06552269 0.06647107
90 0.057947684 0.059403278
100 0.048607938 0.050125778
110 0.0396663 0.040995385
120 0.033573043 0.034936886
130 0.027500706 0.028396493
140 0.021437464 0.02266158
150 0.0180314 0.019531457
160 0.015847037 0.017491423
170 0.0132163875 0.01521089
180 0.011146696 0.013247659
190 0.010126095 0.012114828
17
0 0.3318538 0.33168718
10 0.09298815 0.09110982
20 0.084512725 0.08182567
30 0.07610579 0.07423656
40 0.06269724 0.062103335
50 0.04108754 0.04252241
60 0.024642086 0.026172075
70 0.012557116 0.014497295
80 0.0072117983 0.008695833
90 0.004747449 0.0058615017
100 0.0050046053 0.0059035616
110 0.0030771194 0.0038929968
120 0.0022422515 0.0030012021
130 0.00194

In [69]:
# from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
# event_acc = EventAccumulator('logs/Run_3/60_380_340/testing/events.out.tfevents.1527228830.Andrews-MacBook.local')
# event_acc.Reload()
# # Show all tags in the log file
# print(event_acc.Tags())



{'images': [], 'audio': [], 'histograms': [], 'scalars': [], 'distributions': [], 'tensors': [], 'graph': True, 'meta_graph': False, 'run_metadata': []}
